This notebook contained some quick exploratory analysis of the los angeles crime dataset. Nothing here is required to recreate the final database, but it does give some inisght on why some features were included/excluded.

In [ ]:

import sqlite3
import pandas as pd
pd.set_option('display.max_rows', 142)
conn = sqlite3.connect('crime.db')

cursor = conn.cursor()

create_table_query = '''
    CREATE TABLE IF NOT EXISTS crime_data (
        DR_NO TEXT PRIMARY KEY,               -- Division of Records Number
        "Date_Rptd" DATETIME NOT NULL,        -- Date Reported
        "DATE_OCC" DATETIME NOT NULL,         -- Date Occurred
        "TIME_OCC" TIME NOT NULL,             -- Time Occurred
        "AREA_NAME" TEXT NOT NULL,            -- Area Name
        "Part_1_2" TEXT,                      -- Part 1-2 (Crime Category)
        "Crm_Cd" INTEGER,                     -- Crime Code
        "Crm_Cd_Desc" TEXT NOT NULL,          -- Crime Code Description
        "Vict_Age" INTEGER,                   -- Victim Age
        "Vict_Sex" TEXT,                      -- Victim Sex
        "Vict_Descent" TEXT,                  -- Victim Descent
        "Weapon_Desc" TEXT,                   -- Weapon Description
        "Status_Desc" TEXT,                   -- Status Description
        "Crm_Cd_1" INTEGER,                      -- Crime Code 1
        "Crm_Cd_2" INTEGER,                      -- Crime Code 2
        "Crm_Cd_3" INTEGER,                      -- Crime Code 3
        "Crm_Cd_4" INTEGER,                      -- Crime Code 4
        LOCATION TEXT,                        -- Location of Crime
        LAT REAL,                             -- Latitude
        LON REAL                              -- Longitude
    );
'''


# Execute the create table SQL
cursor.execute(create_table_query)

use_columns = [
    'DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA NAME',
    'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Vict Age',
    'Vict Sex', 'Vict Descent', 'Weapon Desc', 'Status Desc',
    'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 
    'LOCATION',  'LAT', 'LON'
]
csv_file = 'Crime_Data_from_2020_to_Present_20241013.csv'

# Read the CSV into a DataFrame
df = pd.read_csv(csv_file, usecols=use_columns)

# Clean column names
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(' ', '_')
df.rename(columns={'Part_1-2': 'Part_1_2'}, inplace=True)

# Insert the CSV data into the crime_data table
df.to_sql('crime_data', conn, if_exists='append', index=False)

# Commit the transaction and close the connection
conn.commit()
# Query to see crime counts by description to help guide categorization 
query = '''
    SELECT "Crm_Cd_Desc", COUNT(*) as count
    FROM crime_data
    GROUP BY "Crm_Cd_Desc"
    ORDER BY count DESC;
'''

# Execute the query
df_query_results = pd.read_sql_query(query, conn)

# Display the results
df_query_results

In [80]:
query = '''
    SELECT DISTINCT Crm_Cd, Part_1_2
    FROM crime_data
    
'''

# Execute the query
df_query_results = pd.read_sql_query(query, conn)

# Display the results
df_query_results

,Crm_Cd,Part_1_2
0,510,1
1,330,1
2,480,1
3,343,1
4,354,2
5,624,2
6,821,1
7,812,2
8,230,1
9,956,2


In [ ]:


# Function to categorize crime
def categorize_crime(crime_description):
    # Specific category mappings
    property_crimes = {
        'Vehicle Theft': ['VEHICLE - STOLEN', 'VEHICLE, STOLEN', 'BIKE - STOLEN', 'BOAT - STOLEN', 'THEFT FROM MOTOR VEHICLE', 'VEHICLE - ATTEMPT STOLEN'],
        'Burglary': ['BURGLARY', 'BURGLARY FROM VEHICLE', 'BURGLARY, ATTEMPTED', 'BURGLARY FROM VEHICLE, ATTEMPTED'],
        'Theft': ['THEFT', 'PICKPOCKET','SHOPLIFTING', 'EMBEZZLEMENT', 'GRAND THEFT', 'PETTY THEFT', 'THEFT OF IDENTITY'],
        'Vandalism': ['VANDALISM', 'GRAFFITI', 'ARSON']
    }

    violent_crimes = {
        'Assault': ['ASSAULT', 'BATTERY', 'CRIMINAL HOMICIDE', 'CHILD ABUSE', 'BATTERY - SIMPLE ASSAULT', 'ASSAULT WITH DEADLY WEAPON'],
        'Robbery': ['ROBBERY', 'ATTEMPTED ROBBERY'],
        'Weapons': ['BRANDISH WEAPON', 'DISCHARGE FIREARMS', 'SHOTS FIRED', 'FIREARMS']
    }

    sexual_crimes = {
        'Sexual Offenses': ['RAPE', 'LEWD', 'INDECENT EXPOSURE', 'SODOMY/SEXUAL CONTACT', 'SEXUAL PENETRATION', 'PIMPING', 'PANDERING', 'ORAL', 'SEX']
    }

    fraud_crimes = {
        'Fraud': ['THEFT OF IDENTITY', 'EMBEZZLEMENT', 'COUNTERFEIT', 'FRAUD', 'EXTORTION', 'BUNCO', 'FORGERY']
    }

    public_order_crimes = {
        'Public Order': ['COURT','TRESPASSING', 'CONSPIRACY', 'LYNCHING', 'DISORDERLY CONDUCT', 'DISTURBING THE PEACE', 'INCITING RIOT']
    }
    
    # Check in Property Crimes
    for category, crimes in property_crimes.items():
        if any(crime in crime_description for crime in crimes):
            return pd.Series({"Main Category": "Property Crime", "Sub Category": category})
    
    # Check in Violent Crimes
    for category, crimes in violent_crimes.items():
        if any(crime in crime_description for crime in crimes):
            return pd.Series({"Main Category": "Violent Crime", "Sub Category": category})
    
    # Check in Sexual Crimes
    for category, crimes in sexual_crimes.items():
        if any(crime in crime_description for crime in crimes):
            return pd.Series({"Main Category": "Sexual Crime", "Sub Category": category})
    
    # Check in Fraud Crimes
    for category, crimes in fraud_crimes.items():
        if any(crime in crime_description for crime in crimes):
            return pd.Series({"Main Category": "Fraud Crime", "Sub Category": category})
    
    # Check in Public Order Crimes
    for category, crimes in public_order_crimes.items():
        if any(crime in crime_description for crime in crimes):
            return pd.Series({"Main Category": "Public Order Crime", "Sub Category": category})
    
    # If no matches, categorize as 'Other'
    return pd.Series({"Main Category": "Other", "Sub Category": "Other"})


# Query the crime descriptions
query = "SELECT * FROM crime_data"  
df = pd.read_sql_query(query, conn)

# Apply the categorization to the 'Crm_Cd_Desc' column
categories = df['Crm_Cd_Desc'].apply(categorize_crime)

# Concatenate the original dataframe and the categories
df = pd.concat([df, categories], axis=1)

#  Write the categorized data back into  table in SQLite
df.to_sql('crime_data', conn, if_exists='replace', index=False)


# Print the DataFrame with categories


    
    

In [78]:
cursor = conn.cursor()

# Organize the Descent codes to be more general
query = ''' 
SELECT
    DISTINCT Vict_Descent,
    CASE 
        WHEN Vict_Descent IN ('A', 'C', 'D', 'F', 'J', 'K', 'L', 'V', 'Z') THEN 'Asian'
        WHEN Vict_Descent IN ('P', 'S', 'G') THEN 'Pacific Islander'
        WHEN Vict_Descent = 'H' THEN 'Hispanic/Latin/Mexican'
        WHEN Vict_Descent = 'B' THEN 'Black'
        WHEN Vict_Descent = 'W' THEN 'White'
        WHEN Vict_Descent = 'I' THEN 'American Indian/Alaskan Native'
        WHEN Vict_Descent = 'X' THEN 'Unknown'
        ELSE 'Other' 
    END AS Descent_Category
FROM 
    crime_data
'''

# Execute the query
descent_by_area = pd.read_sql_query(query, conn)

descent_by_area

,Vict_Descent,Descent_Category
0,O,Other
1,X,Unknown
2,H,Hispanic/Latin/Mexican
3,B,Black
4,W,White
5,None,Other
6,A,Asian
7,K,Asian
8,C,Asian
9,J,Asian


In [73]:
cursor = conn.cursor()

# Organize the Descent codes to be more general
query = ''' 
SELECT
    AREA_NAME,
    CASE 
        WHEN Vict_Descent IN ('C', 'D', 'F', 'J', 'K', 'L', 'V', 'Z') THEN 'Asian'
        WHEN Vict_Descent IN ('P', 'S', 'G') THEN 'Pacific Islander'
        WHEN Vict_Descent = 'H' THEN 'Hispanic/Latin/Mexican'
        WHEN Vict_Descent = 'B' THEN 'Black'
        WHEN Vict_Descent = 'W' THEN 'White'
        WHEN Vict_Descent = 'I' THEN 'American Indian/Alaskan Native'
        WHEN Vict_Descent = 'X' THEN 'Unknown'
        ELSE 'Other' 
    END AS Descent_Category, COUNT(*) AS count
FROM 
    crime_data
GROUP BY 
    AREA_NAME, Descent_Category
HAVING 
    Descent_Category NOT IN ('Unknown', 'Other')
ORDER BY AREA_NAME, count DESC
'''

# Execute the query
descent_by_area = pd.read_sql_query(query, conn)

# Display the results
descent_by_area.to_csv('data/crime_volume_by_area_and_descent.csv',index = False)

In [63]:
conn = sqlite3.connect('crime.db')

cursor = conn.cursor()

# Query to see some crime statistics by AREA_NAME
query = '''
    SELECT 
    AREA_NAME, AVG(Vict_Age), AVG(LAT), AVG(LON), COUNT(*) as count,
    SUM(CASE WHEN Vict_Sex = 'M' THEN 1 ELSE 0 END) AS Male_Count,
    SUM(CASE WHEN Vict_Sex = 'F' THEN 1 ELSE 0 END) AS Female_Count
    FROM crime_data
    GROUP BY "AREA_NAME"
'''

# Execute the query
crime_by_area = pd.read_sql_query(query, conn)

# Display the results
crime_by_area.to_csv('data/crime_volume_by_area.csv',index = False)

Add crime descriptions for Crm Cd 2,3 and 4.

In [53]:
conn = sqlite3.connect('crime.db')  # Replace with your actual database filename
cursor = conn.cursor()

cursor.execute('''
    CREATE TEMPORARY TABLE crime_code_mapping AS
    SELECT DISTINCT Crm_Cd, Crm_Cd_Desc FROM crime_data;
''')

In [54]:
query = '''
SELECT  
    cd.Crm_Cd_2,
    mapping2.Crm_Cd_Desc AS Crm_Cd_2_Desc
FROM 
    crime_data cd
LEFT JOIN 
    crime_code_mapping mapping2 ON cd.Crm_Cd_2 = mapping2.Crm_Cd_1
    '''
    
query2 = '''
SELECT DISTINCT 
    CAST(Crm_Cd AS REAL) AS Crm_Cd,
    Crm_Cd_1
FROM crime_data
WHERE (Crm_Cd - Crm_Cd_1) != 0
'''
# Execute the query
crime_with_desc = pd.read_sql_query(query2, conn)

# Display the results
crime_with_desc 

,Crm_Cd,Crm_Cd_1
0,821.0,812.0
1,820.0,812.0
2,815.0,812.0
3,812.0,760.0
4,510.0,341.0
...,...,...
163,910.0,434.0
164,810.0,624.0
165,510.0,626.0
166,886.0,434.0


- It looks like Crime Code 1 ad Crime Code 2 are not identical, contrary to what data.lacity.org said on the column description. 
- The inconsistencies do not follow any pattern and do not look like a data entry issue

In [57]:


# Query to join the crime_data table with the mapping table
query = '''
SELECT 
    CAST(cd.Crm_Cd AS REAL) AS Crm_Cd_1,
    mapping1.Crm_Cd_Desc AS Crm_Cd_1_Desc,
    
    cd.Crm_Cd_2,
    mapping2.Crm_Cd_Desc AS Crm_Cd_2_Desc,
    
    cd.Crm_Cd_3,
    mapping3.Crm_Cd_Desc AS Crm_Cd_3_Desc,
    
    cd.Crm_Cd_4,
    mapping4.Crm_Cd_Desc AS Crm_Cd_4_Desc
FROM 
    crime_data cd
LEFT JOIN 
    crime_code_mapping mapping1 ON cd.Crm_Cd_1 = mapping1.Crm_Cd
LEFT JOIN 
    crime_code_mapping mapping2 ON cd.Crm_Cd_2 = mapping2.Crm_Cd
LEFT JOIN 
    crime_code_mapping mapping3 ON cd.Crm_Cd_3 = mapping3.Crm_Cd
LEFT JOIN 
    crime_code_mapping mapping4 ON cd.Crm_Cd_4 = mapping4.Crm_Cd;
'''


# Execute the query
crime_with_desc = pd.read_sql_query(query, conn)

# Display the results
crime_with_desc 

,Crm_Cd_1,Crm_Cd_1_Desc,Crm_Cd_2,Crm_Cd_2_Desc,Crm_Cd_3,Crm_Cd_3_Desc,Crm_Cd_4,Crm_Cd_4_Desc
0,510.0,VEHICLE - STOLEN,998.0,None,NaN,None,NaN,None
1,330.0,BURGLARY FROM VEHICLE,998.0,None,NaN,None,NaN,None
2,480.0,BIKE - STOLEN,NaN,None,NaN,None,NaN,None
3,343.0,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),NaN,None,NaN,None,NaN,None
4,354.0,THEFT OF IDENTITY,NaN,None,NaN,None,NaN,None
...,...,...,...,...,...,...,...,...
982633,341.0,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",NaN,None,NaN,None,NaN,None
982634,510.0,VEHICLE - STOLEN,NaN,None,NaN,None,NaN,None
982635,745.0,VANDALISM - MISDEAMEANOR ($399 OR UNDER),NaN,None,NaN,None,NaN,None
982636,230.0,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",NaN,None,NaN,None,NaN,None


In [58]:
crime_with_desc.count()

Crm_Cd_1         982638
Crm_Cd_1_Desc    982617
Crm_Cd_2          68875
Crm_Cd_2_Desc     13477
Crm_Cd_3           2311
Crm_Cd_3_Desc       246
Crm_Cd_4             64
Crm_Cd_4_Desc         5
dtype: int64

- A substantial amount of Crm_Cd_2, Crm_Cd_3, Crm_Cd_4 are missing descriptions, indicating that Crm_Cd and Crm_Cd_Desc only make up a fraction of the total crime codes and descriptions. 
- Additionally, the pdf that details every Crm Cd and Crm Cd Description from data.lacity.org is missing the distinct codes from  Crm_Cd_2, Crm_Cd_3, Crm_Cd_4. I could not find any other sources for these codes, so we unfortunately will have to ignore the columns Crm_Cd_1, Crm_Cd_2, Crm_Cd_3, and Crm_Cd_4

In [108]:

# Find the datatype for each row in 'mixed_column'
df = pd.read_csv('/Users/connerkhudaverdyan/Desktop/Projects/Crime_Project/data/crime_data_processed.csv')